# Elliptical, parabolic, and hyperbolic orbits using the Euler method

In the simple_two_body.ipynb notebook I implemented circular orbits with arbitary starting positions and confirmed that the truncation error converged linearly with timestep for the Euler method.  

In this notebook I plan to implement hyperbolic, elliptical, and parabolic orbits using the Euler method. Parabolic orbits are marginally bound, hyperbolic orbits are unbound, and elliptical orbits are bound but oblong. All orbits should have the center of mass at one focus for both stars orbit and be symmetric between the two stars. 

Elliptical orbits should be elongated away from the other star. 

Hyperbolic and parabolic orbits should open away from the other star. 

All orbits should conserve energy and angular momentum. Later this could be enforced. I will probably not do that in this notebook. 

It would be meaningless to evaluate the L0 error for a hyperbolic or parabolic orbit since there is no cycle and the L0 error is found after one or more cycles when the system returns to its starting point. However, it would be possible to evaluate it for the elliptical orbit system, and I intend to check that the convergence of the L0 error with dt is linear as it should be for the Euler method. 

Because the convergence can be tested for ellipses only and because ellipses are closed so make it clear if they are not working correctly graphically, I will begin with ellipses then proceed to parabolas and hyperbolas. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

In [2]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

In [3]:
import math
def InitialDataEqualMassConic(ecc,angle,initmass):
    orbitalangle=angle
    phi=np.array([orbitalangle,math.pi+orbitalangle])
    orbitalradius=100*np.ones(2) #semimajor axis
    eccentricity=ecc
    print(eccentricity)
    mass=np.ones(2)
    masses=initmass*mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [72]:
import random,numpy as np
initdateqellipse=InitialDataEqualMassConic(0.1,0.,1.)
print(initdateqellipse)

0.1
(array([0.        , 3.14159265]), 0.0, array([100., 100.]), 0.1, array([1., 1.]))


In [111]:
def getxyuveqconic(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    print("initdat", orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    #fix x0 y0 at one star, disregard initial data, use orbital radius as separation between stars
    #this is consistant with choice in previous part
    cosphi=np.cos(phi)
    sinphi=np.sin(phi)
    coordsep=orbitalradius #/2.
    print("coordsep",coordsep)
    #x0=orbitalradius/2.*np.cos(phi)*metersperAU
    #y0=orbitalradius/2.*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            #x0[count]=orbitalradius[count]/2.
            #y0[count]=0
            cosphi[count]=1.0
            sinphi[count]=0.0
        if phi0==math.pi:
            cosphi[count]=-1.0
            sinphi[count]=0.0
            print("pi")
            #x0[count]=-orbitalradius[count]/2.
            #y0[count]=0
        if phi0==math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=1.0
            #x0[count]=0
            #y0[count]=orbitalradius[count]/2.
        if phi0==3*math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=-1.0
            #x0[count]=0
            #y0[count]=-orbitalradius[count]/2.
        count+=1
    x0=coordsep*cosphi
    y0=coordsep*sinphi
    #x0[1]=0.0
    #y0[1]=0.0
    z0=np.zeros(2)
    print(x0)
    print(y0)
    v=np.zeros(2)
    a=np.zeros(2)
    ax0=0.
    ay0=0.
    az0=0.
    vx0=0.
    vy0=0.
    vz0=0.
    print(eccentricity)
    if eccentricity<1 and eccentricity>0:
        #elliptical
        orbitalr=orbitalradius[0]/2
        E=Gconstant*masses[0]*masses[1]/(2*orbitalr) #1/2*mv^2-GMm/r #orbitalradius is correct here r1+r2
        coordsep=orbitalr*(1.+eccentricity) #starts at apastron
        starsep=2.*coordsep
        Eapastron=Gconstant*masses[0]*masses[1]/(starsep)
        Fapastron=Eapastron/(starsep)
        print("E,F",E,Fapastron,2*orbitalr,starsep)
        x0=coordsep*cosphi
        y0=coordsep*sinphi
        v=np.zeros(2)
        v=np.sqrt(2.*(2*Eapastron-E)/masses) #virial theorem (not the viral theorem, the VEE-REE-AHL theoerem, a thermodynamics theorem about stellar dynamics and energy among other things)
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        a=Fapastron/masses
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    elif eccentricity[0]==0.0: #circular
        #start at perihelion for both (eliptical, doesn't generalize to three body)
        #actually start with circular orbit
        ux0=np.zeros(2) #*149597870700
        #centrepital force balances gravitational force
        metersperAU=1 #natural units
        #G=1
        Gconstant=1
        #Fcentripital=mass1*v**2/rphys
        #centF=accel
        #Faccel=G*m1*m2/r^2
        r0=2.*orbitalradius #Mystery factor of 2
        print("r0", r0)
        v=np.zeros(2)
        for i in np.arange(2):
            v[i]=np.sqrt(Gconstant*masses[(i+1)%2]/np.abs(r0[i]))
        print(v)
        #r0=orbitalradius #np.sqrt(x0**2+y0**2)
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        for i in np.arange(2):
            a[i]=Gconstant*masses[(i+1)%2]/(r0[i]**2)
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    
    
    return masses,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [112]:
xyuvaeqconic=getxyuveqconic(initdateqellipse)
print(xyuvaeqconic)

initdat [100. 100.] [0.         3.14159265] [ 1. -1.] [0.0000000e+00 1.2246468e-16]
coordsep [100. 100.]
zero
pi
[ 100. -100.]
[0. 0.]
0.1
E,F 0.01 8.264462809917353e-05 100.0 110.00000000000001
(array([1., 1.]), array([ 55., -55.]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([ 0.12792043, -0.12792043]), array([0., 0.]), array([-8.26446281e-05,  8.26446281e-05]), array([-0., -0.]), array([0., 0.]))


In [113]:
def timestep(step,t,dt,mtotal, mass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vxi))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vyi))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=np.zeros(np.size(axi))
    ayii=np.zeros(np.size(ayi))
    azii=np.zeros(np.size(azi))
    
    #This is redundant in the center of mass system
    Gconstant=6.408*10**-11
    Gconstant=1
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                #print(rreljk)
                axii[j]-=Gconstant*mass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]-=Gconstant*mass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]-=Gconstant*mass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        vxii[m] = vxi[m] + dt*axi[m]
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        
        #print(vyii[m],vyi[m],ayi[m],dt)
        #print(yii[m],yi[m],vyi[m],dt)
    
    
    return mass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [114]:
dt=1 #*31556926 #seconds per year
numsteps=10000
mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeqconic
print(x)
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
t=0.0
masstotal=1.
for i in np.arange(1,numsteps):
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    xcoord1.append(x[0])
    xcoord2.append(x[1])
    ycoord1.append(y[0])
    ycoord2.append(y[1])
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[ 55. -55.]
[ 55. -55.] [ 0.12792043 -0.12792043] [-8.26446281e-05  8.26446281e-05] [ 0.12792043 -0.12792043] [-8.26446281e-05  8.26446281e-05] [0. 0.]
[ 54.99991736 -54.99991736] [ 0.25584086 -0.25584086] [-0.00016529  0.00016529] [ 0.12792043 -0.12792043] [-8.26439575e-05  8.26439575e-05] [-1.92215465e-07  1.92215465e-07]
[ 54.99975207 -54.99975207] [ 0.38376129 -0.38376129] [-0.00024793  0.00024793] [ 0.12792024 -0.12792024] [-8.26421941e-05  8.26421941e-05] [-3.84423305e-07  3.84423305e-07]
[ 54.99950413 -54.99950413] [ 0.51168153 -0.51168153] [-0.00033058  0.00033058] [ 0.12791985 -0.12791985] [-8.26393381e-05  8.26393381e-05] [-5.76616762e-07  5.76616762e-07]
[ 54.99917356 -54.99917356] [ 0.63960138 -0.63960138] [-0.00041321  0.00041321] [ 0.12791928 -0.12791928] [-8.26353896e-05  8.26353896e-05] [-7.68788792e-07  7.68788792e-07]
[ 54.99876034 -54.99876034] [ 0.76752066 -0.76752066] [-0.00049585  0.00049585] [ 0.12791851 -0.12791851] [-8.26303488e-05  8.26303488e-05] [-9.60932351

[ 53.40503042 -53.40503042] [ 25.47629921 -25.47629921] [-0.01532211  0.01532211] [ 0.12450549 -0.12450549] [-6.45962476e-05  6.45962476e-05] [-3.06555587e-05  3.06555587e-05]
[ 53.38970831 -53.38970831] [ 25.6008047 -25.6008047] [-0.01538671  0.01538671] [ 0.12447483 -0.12447483] [-6.44478046e-05  6.44478046e-05] [-3.07441367e-05  3.07441367e-05]
[ 53.3743216 -53.3743216] [ 25.72527953 -25.72527953] [-0.01545115  0.01545115] [ 0.12444409 -0.12444409] [-6.42991605e-05  6.42991605e-05] [-3.08319768e-05  3.08319768e-05]
[ 53.35887045 -53.35887045] [ 25.84972361 -25.84972361] [-0.01551545  0.01551545] [ 0.12441325 -0.12441325] [-6.41503204e-05  6.41503204e-05] [-3.09190801e-05  3.09190801e-05]
[ 53.34335499 -53.34335499] [ 25.97413687 -25.97413687] [-0.0155796  0.0155796] [ 0.12438234 -0.12438234] [-6.40012898e-05  6.40012898e-05] [-3.10054474e-05  3.10054474e-05]
[ 53.32777539 -53.32777539] [ 26.0985192 -26.0985192] [-0.0156436  0.0156436] [ 0.12435133 -0.12435133] [-6.38520739e-05  6.38

[ 50.69897424 -50.69897424] [ 42.57981002 -42.57981002] [-0.02290129  0.02290129] [ 0.11965686 -0.11965686] [-4.38110462e-05  4.38110462e-05] [-3.66749793e-05  3.66749793e-05]
[ 50.67607296 -50.67607296] [ 42.69946688 -42.69946688] [-0.0229451  0.0229451] [ 0.11962019 -0.11962019] [-4.36735637e-05  4.36735637e-05] [-3.66794807e-05  3.66794807e-05]
[ 50.65312786 -50.65312786] [ 42.81908707 -42.81908707] [-0.02298877  0.02298877] [ 0.11958351 -0.11958351] [-4.35363472e-05  4.35363472e-05] [-3.6683561e-05  3.6683561e-05]
[ 50.63013909 -50.63013909] [ 42.93867058 -42.93867058] [-0.02303231  0.02303231] [ 0.11954683 -0.11954683] [-4.33993981e-05  4.33993981e-05] [-3.66872232e-05  3.66872232e-05]
[ 50.60710678 -50.60710678] [ 43.0582174 -43.0582174] [-0.02307571  0.02307571] [ 0.11951014 -0.11951014] [-4.32627177e-05  4.32627177e-05] [-3.66904697e-05  3.66904697e-05]
[ 50.58403107 -50.58403107] [ 43.17772754 -43.17772754] [-0.02311897  0.02311897] [ 0.11947345 -0.11947345] [-4.31263073e-05  

[ 47.94519724 -47.94519724] [ 55.52334188 -55.52334188] [-0.02695916  0.02695916] [ 0.11567176 -0.11567176] [-3.04653461e-05  3.04653461e-05] [-3.51873665e-05  3.51873665e-05]
[ 47.91823808 -47.91823808] [ 55.63901364 -55.63901364] [-0.02698962  0.02698962] [ 0.11563658 -0.11563658] [-3.03612747e-05  3.03612747e-05] [-3.51601315e-05  3.51601315e-05]
[ 47.89124846 -47.89124846] [ 55.75465022 -55.75465022] [-0.02701998  0.02701998] [ 0.11560142 -0.11560142] [-3.02575149e-05  3.02575149e-05] [-3.5132725e-05  3.5132725e-05]
[ 47.86422847 -47.86422847] [ 55.87025163 -55.87025163] [-0.02705024  0.02705024] [ 0.11556628 -0.11556628] [-3.01540662e-05  3.01540662e-05] [-3.5105149e-05  3.5105149e-05]
[ 47.83717823 -47.83717823] [ 55.98581792 -55.98581792] [-0.0270804  0.0270804] [ 0.11553118 -0.11553118] [-3.00509283e-05  3.00509283e-05] [-3.5077405e-05  3.5077405e-05]
[ 47.81009784 -47.81009784] [ 56.1013491 -56.1013491] [-0.02711045  0.02711045] [ 0.1154961 -0.1154961] [-2.9948101e-05  2.99481

[ 43.27539891 -43.27539891] [ 73.71521967 -73.71521967] [-0.0307237  0.0307237] [ 0.11043502 -0.11043502] [-1.73829221e-05  1.73829221e-05] [-2.95446962e-05  2.95446962e-05]
[ 43.24467521 -43.24467521] [ 73.82565469 -73.82565469] [-0.03074108  0.03074108] [ 0.11040547 -0.11040547] [-1.73220676e-05  1.73220676e-05] [-2.95063721e-05  2.95063721e-05]
[ 43.21393412 -43.21393412] [ 73.93606017 -73.93606017] [-0.03075841  0.03075841] [ 0.11037597 -0.11037597] [-1.72614283e-05  1.72614283e-05] [-2.94680498e-05  2.94680498e-05]
[ 43.18317572 -43.18317572] [ 74.04643613 -74.04643613] [-0.03077567  0.03077567] [ 0.1103465 -0.1103465] [-1.72010033e-05  1.72010033e-05] [-2.94297301e-05  2.94297301e-05]
[ 43.15240005 -43.15240005] [ 74.15678263 -74.15678263] [-0.03079287  0.03079287] [ 0.11031707 -0.11031707] [-1.71407921e-05  1.71407921e-05] [-2.93914133e-05  2.93914133e-05]
[ 43.12160718 -43.12160718] [ 74.2670997 -74.2670997] [-0.03081001  0.03081001] [ 0.11028768 -0.11028768] [-1.70807939e-05  

...

[-257.1832768  257.1832768] [ 825.88164451 -825.88164451] [-0.03402933  0.03402933] [ 0.08187762 -0.08187762] [ 9.93598773e-08 -9.93598773e-08] [-3.19080706e-07  3.19080706e-07]
[-257.21730613  257.21730613] [ 825.96352213 -825.96352213] [-0.03402923  0.03402923] [ 0.08187731 -0.08187731] [ 9.93425992e-08 -9.93425992e-08] [-3.19014635e-07  3.19014635e-07]
[-257.25133536  257.25133536] [ 826.04539944 -826.04539944] [-0.03402913  0.03402913] [ 0.08187699 -0.08187699] [ 9.93253255e-08 -9.93253255e-08] [-3.18948584e-07  3.18948584e-07]
[-257.2853645  257.2853645] [ 826.12727643 -826.12727643] [-0.03402903  0.03402903] [ 0.08187667 -0.08187667] [ 9.93080562e-08 -9.93080562e-08] [-3.18882555e-07  3.18882555e-07]
[-257.31939353  257.31939353] [ 826.20915309 -826.20915309] [-0.03402893  0.03402893] [ 0.08187635 -0.08187635] [ 9.92907914e-08 -9.92907914e-08] [-3.18816545e-07  3.18816545e-07]
[-257.35342246  257.35342246] [ 826.29102944 -826.29102944] [-0.03402883  0.03402883] [ 0.08187603 -0.08

[-261.70831221  261.70831221] [ 836.76859279 -836.76859279] [-0.03401627  0.03401627] [ 0.08183576 -0.08183576] [ 9.71001423e-08 -9.71001423e-08] [-3.10471469e-07  3.10471469e-07]
[-261.74232847  261.74232847] [ 836.85042854 -836.85042854] [-0.03401617  0.03401617] [ 0.08183545 -0.08183545] [ 9.70834396e-08 -9.70834396e-08] [-3.10408074e-07  3.10408074e-07]
[-261.77634464  261.77634464] [ 836.93226399 -836.93226399] [-0.03401607  0.03401607] [ 0.08183514 -0.08183514] [ 9.70667411e-08 -9.70667411e-08] [-3.10344698e-07  3.10344698e-07]
[-261.81036071  261.81036071] [ 837.01409913 -837.01409913] [-0.03401597  0.03401597] [ 0.08183483 -0.08183483] [ 9.70500468e-08 -9.70500468e-08] [-3.10281341e-07  3.10281341e-07]
[-261.84437669  261.84437669] [ 837.09593396 -837.09593396] [-0.03401588  0.03401588] [ 0.08183452 -0.08183452] [ 9.70333567e-08 -9.70333567e-08] [-3.10218005e-07  3.10218005e-07]
[-261.87839257  261.87839257] [ 837.17776848 -837.17776848] [-0.03401578  0.03401578] [ 0.08183421 -

[-266.02760663  266.02760663] [ 847.15927106 -847.15927106] [-0.03400407  0.03400407] [ 0.08179683 -0.08179683] [ 9.50122985e-08 -9.50122985e-08] [-3.02574111e-07  3.02574111e-07]
[-266.06161069  266.06161069] [ 847.24106789 -847.24106789] [-0.03400397  0.03400397] [ 0.08179653 -0.08179653] [ 9.49961223e-08 -9.49961223e-08] [-3.02513137e-07  3.02513137e-07]
[-266.09561467  266.09561467] [ 847.32286441 -847.32286441] [-0.03400388  0.03400388] [ 0.08179623 -0.08179623] [ 9.49799502e-08 -9.49799502e-08] [-3.02452181e-07  3.02452181e-07]
[-266.12961854  266.12961854] [ 847.40466064 -847.40466064] [-0.03400378  0.03400378] [ 0.08179592 -0.08179592] [ 9.49637821e-08 -9.49637821e-08] [-3.02391244e-07  3.02391244e-07]
[-266.16362232  266.16362232] [ 847.48645656 -847.48645656] [-0.03400369  0.03400369] [ 0.08179562 -0.08179562] [ 9.49476181e-08 -9.49476181e-08] [-3.02330325e-07  3.02330325e-07]
[-266.19762601  266.19762601] [ 847.56825218 -847.56825218] [-0.03400359  0.03400359] [ 0.08179532 -

[-270.92322179  270.92322179] [ 858.93492869 -858.93492869] [-0.03399055  0.03399055] [ 0.08175388 -0.08175388] [ 9.27241019e-08 -9.27241019e-08] [-2.93981377e-07  2.93981377e-07]
[-270.95721234  270.95721234] [ 859.01668257 -859.01668257] [-0.03399046  0.03399046] [ 0.08175358 -0.08175358] [ 9.27084971e-08 -9.27084971e-08] [-2.93923001e-07  2.93923001e-07]
[-270.99120279  270.99120279] [ 859.09843615 -859.09843615] [-0.03399036  0.03399036] [ 0.08175329 -0.08175329] [ 9.26928962e-08 -9.26928962e-08] [-2.93864642e-07  2.93864642e-07]
[-271.02519316  271.02519316] [ 859.18018944 -859.18018944] [-0.03399027  0.03399027] [ 0.081753 -0.081753] [ 9.26772991e-08 -9.26772991e-08] [-2.938063e-07  2.938063e-07]
[-271.05918343  271.05918343] [ 859.26194244 -859.26194244] [-0.03399018  0.03399018] [ 0.0817527 -0.0817527] [ 9.2661706e-08 -9.2661706e-08] [-2.93747977e-07  2.93747977e-07]
[-271.09317361  271.09317361] [ 859.34369514 -859.34369514] [-0.03399009  0.03399009] [ 0.08175241 -0.08175241] 

[-277.00606746  277.00606746] [ 873.56424364 -873.56424364] [-0.0339742  0.0339742] [ 0.08170217 -0.08170217] [ 8.99914406e-08 -8.99914406e-08] [-2.83804593e-07  2.83804593e-07]
[-277.04004166  277.04004166] [ 873.64594581 -873.64594581] [-0.03397411  0.03397411] [ 0.08170189 -0.08170189] [ 8.99765102e-08 -8.99765102e-08] [-2.83749243e-07  2.83749243e-07]
[-277.07401576  277.07401576] [ 873.7276477 -873.7276477] [-0.03397402  0.03397402] [ 0.0817016 -0.0817016] [ 8.99615834e-08 -8.99615834e-08] [-2.8369391e-07  2.8369391e-07]
[-277.10798978  277.10798978] [ 873.8093493 -873.8093493] [-0.03397393  0.03397393] [ 0.08170132 -0.08170132] [ 8.99466604e-08 -8.99466604e-08] [-2.83638593e-07  2.83638593e-07]
[-277.14196371  277.14196371] [ 873.89105062 -873.89105062] [-0.03397384  0.03397384] [ 0.08170103 -0.08170103] [ 8.99317409e-08 -8.99317409e-08] [-2.83583292e-07  2.83583292e-07]
[-277.17593754  277.17593754] [ 873.97275165 -873.97275165] [-0.03397375  0.03397375] [ 0.08170075 -0.08170075

In [115]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="dt=1, Euler Method, Equal Mass System, Ellipse", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(xcoord1,ycoord1, size=1, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="M")
p.circle(xcoord2,ycoord2, size=1, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="mu")

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...